### Initial setup

1. Import libraries
 
2. Read files

3. Split to train, test, val

4. Fill in gaps

#### Imports

In [56]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier   
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, f1_score

#### Load data

In [57]:
# Paths and constants
TRAIN_FILE_PATH = "./data/train.csv"
VAL_FILE_PATH = "./data/test.csv"
TEST_SIZE = 0.2
RANDOM_STATE = 42

# Read files
train_df = pd.read_csv(TRAIN_FILE_PATH)
val_df = pd.read_csv(VAL_FILE_PATH)

# Split to train, test, val
train_df, test_df = train_test_split(train_df, test_size=TEST_SIZE, random_state=RANDOM_STATE)  

#### Sanitize data

In [58]:
# Sanitize data
## Split Cabin info
train_df[["cabin_letter", "cabin_id", "cabin_side"]] = train_df["Cabin"].str.split("/", expand=True)
test_df[["cabin_letter", "cabin_id", "cabin_side"]] = test_df["Cabin"].str.split("/", expand=True)
val_df[["cabin_letter", "cabin_id", "cabin_side"]] = val_df["Cabin"].str.split("/", expand=True)

## Drop not used columns
train_df = train_df.drop(columns=["PassengerId", "Name", "Cabin", "cabin_id", "cabin_side"])
test_df = test_df.drop(columns=["PassengerId", "Name", "Cabin", "cabin_id", "cabin_side"])
val_df = val_df.drop(columns=["PassengerId", "Name", "Cabin", "cabin_id", "cabin_side"])

#### Fill in the gaps

In [59]:
categorical_cols = ['HomePlanet', 'Destination', 'cabin_letter']
binary_cols = ['VIP', 'CryoSleep', 'Transported']


In [60]:
## Fill in gaps for cabin_letter
### All possible values for cabin_letter
train_cabin_letter_values = train_df["cabin_letter"].unique()
train_most_frequent = train_df['cabin_letter'].value_counts().idxmax()
print(f"train_cabin_letter_values: {train_cabin_letter_values} most frequent: {train_most_frequent}")

test_cabin_letter_values = test_df["cabin_letter"].unique()
test_most_frequent = test_df['cabin_letter'].value_counts().idxmax()
print(f"test_cabin_letter_values: {test_cabin_letter_values} most frequent: {test_most_frequent}")

val_cabin_letter_values = val_df["cabin_letter"].unique()
val_most_frequent = val_df['cabin_letter'].value_counts().idxmax()
print(f"val_cabin_letter_values: {val_cabin_letter_values} most frequent: {val_most_frequent}")

cabin_letter_mapping = {
    "A": 1,
    "B": 2,
    "C": 3,
    "D": 4,
    "E": 5,
    "F": 6,
    "G": 7,
    "T": 8
}

### Fill all missing values with most frequent value
train_df['cabin_letter'] = train_df['cabin_letter'].fillna(train_most_frequent)
test_df['cabin_letter'] = test_df['cabin_letter'].fillna(test_most_frequent)
val_df['cabin_letter'] = val_df['cabin_letter'].fillna(val_most_frequent)

### Map cabin_letter to number
train_df['cabin_letter'] = train_df['cabin_letter'].map(cabin_letter_mapping)
test_df['cabin_letter'] = test_df['cabin_letter'].map(cabin_letter_mapping)
val_df['cabin_letter'] = val_df['cabin_letter'].map(cabin_letter_mapping)



train_cabin_letter_values: [nan 'F' 'C' 'G' 'B' 'E' 'D' 'A' 'T'] most frequent: F
test_cabin_letter_values: ['F' 'G' 'D' 'A' 'E' 'B' 'C' nan] most frequent: F
val_cabin_letter_values: ['G' 'F' 'C' 'B' 'D' 'E' nan 'A' 'T'] most frequent: F


In [61]:
train_cabin_letter_values = train_df["Destination"].unique()
train_most_frequent = train_df['Destination'].value_counts().idxmax()
print(f"train_cabin_letter_values: {train_cabin_letter_values} most frequent: {train_most_frequent}")

test_cabin_letter_values = test_df["Destination"].unique()
test_most_frequent = test_df['Destination'].value_counts().idxmax()
print(f"test_cabin_letter_values: {test_cabin_letter_values} most frequent: {test_most_frequent}")

val_cabin_letter_values = val_df["Destination"].unique()
val_most_frequent = val_df['Destination'].value_counts().idxmax()
print(f"val_cabin_letter_values: {val_cabin_letter_values} most frequent: {val_most_frequent}")

destination_mapping = {
    "TRAPPIST-1e": 1,
    "55 Cancri e": 2,
    "PSO J318.5-22": 3
}

### Fill all missing values with most frequent value
train_df['Destination'] = train_df['Destination'].fillna(train_most_frequent)
test_df['Destination'] = test_df['Destination'].fillna(test_most_frequent)
val_df['Destination'] = val_df['Destination'].fillna(val_most_frequent)

### Map Destination to number
train_df['Destination'] = train_df['Destination'].map(destination_mapping)
test_df['Destination'] = test_df['Destination'].map(destination_mapping)
val_df['Destination'] = val_df['Destination'].map(destination_mapping)


train_cabin_letter_values: ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' nan] most frequent: TRAPPIST-1e
test_cabin_letter_values: ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' nan] most frequent: TRAPPIST-1e
val_cabin_letter_values: ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' nan] most frequent: TRAPPIST-1e


In [62]:
train_cabin_letter_values = train_df["HomePlanet"].unique()
train_most_frequent = train_df['HomePlanet'].value_counts().idxmax()    
print(f"train_cabin_letter_values: {train_cabin_letter_values} most frequent: {train_most_frequent}")

test_cabin_letter_values = test_df["HomePlanet"].unique()
test_most_frequent = test_df['HomePlanet'].value_counts().idxmax()
print(f"test_cabin_letter_values: {test_cabin_letter_values} most frequent: {test_most_frequent}")

val_cabin_letter_values = val_df["HomePlanet"].unique()
val_most_frequent = val_df['HomePlanet'].value_counts().idxmax()
print(f"val_cabin_letter_values: {val_cabin_letter_values} most frequent: {val_most_frequent}")

home_planet_mapping = {
    "Earth": 1,
    "Europa": 2,
    "Mars": 3
}

### Fill all missing values with most frequent value
train_df['HomePlanet'] = train_df['HomePlanet'].fillna(train_most_frequent)
test_df['HomePlanet'] = test_df['HomePlanet'].fillna(test_most_frequent)
val_df['HomePlanet'] = val_df['HomePlanet'].fillna(val_most_frequent)

### Map HomePlanet to number
train_df['HomePlanet'] = train_df['HomePlanet'].map(home_planet_mapping)
test_df['HomePlanet'] = test_df['HomePlanet'].map(home_planet_mapping)
val_df['HomePlanet'] = val_df['HomePlanet'].map(home_planet_mapping)

train_cabin_letter_values: ['Earth' 'Europa' 'Mars' nan] most frequent: Earth
test_cabin_letter_values: ['Mars' 'Earth' 'Europa' nan] most frequent: Earth
val_cabin_letter_values: ['Earth' 'Europa' 'Mars' nan] most frequent: Earth


In [63]:
### Convert binary columns to numbers
binary_mapping = {
    True: 1,
    False: 0
}

In [64]:
## VIP
train_vip_most_frequent = train_df['VIP'].value_counts().idxmax()
print(f"train_vip_most_frequent: {train_vip_most_frequent}")

test_vip_most_frequent = test_df['VIP'].value_counts().idxmax()
print(f"test_vip_most_frequent: {test_vip_most_frequent}")

val_vip_most_frequent = val_df['VIP'].value_counts().idxmax()
print(f"val_vip_most_frequent: {val_vip_most_frequent}")

### Fill all missing values with most frequent value
train_df['VIP'] = train_df['VIP'].fillna(train_vip_most_frequent)
test_df['VIP'] = test_df['VIP'].fillna(test_vip_most_frequent)
val_df['VIP'] = val_df['VIP'].fillna(val_vip_most_frequent)

### Map VIP to number
train_df['VIP'] = train_df['VIP'].map(binary_mapping)
test_df['VIP'] = test_df['VIP'].map(binary_mapping)
val_df['VIP'] = val_df['VIP'].map(binary_mapping)


train_vip_most_frequent: False
test_vip_most_frequent: False
val_vip_most_frequent: False


/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/4135553618.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['VIP'] = train_df['VIP'].fillna(train_vip_most_frequent)
/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/4135553618.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['VIP'] = test_df['VIP'].fillna(test_vip_most_frequent)
/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/4135553618.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated an

In [65]:
## CryoSleep
train_cryo_sleep_most_frequent = train_df['CryoSleep'].value_counts().idxmax()
print(f"train_cryo_sleep_most_frequent: {train_cryo_sleep_most_frequent}")

test_cryo_sleep_most_frequent = test_df['CryoSleep'].value_counts().idxmax()
print(f"test_cryo_sleep_most_frequent: {test_cryo_sleep_most_frequent}")

val_cryo_sleep_most_frequent = val_df['CryoSleep'].value_counts().idxmax()
print(f"val_cryo_sleep_most_frequent: {val_cryo_sleep_most_frequent}")


### Fill all missing values with most frequent value
train_df['CryoSleep'] = train_df['CryoSleep'].fillna(train_cryo_sleep_most_frequent)
test_df['CryoSleep'] = test_df['CryoSleep'].fillna(test_cryo_sleep_most_frequent)
val_df['CryoSleep'] = val_df['CryoSleep'].fillna(val_cryo_sleep_most_frequent)

train_df['CryoSleep'] = train_df['CryoSleep'].map(binary_mapping)
test_df['CryoSleep'] = test_df['CryoSleep'].map(binary_mapping)
val_df['CryoSleep'] = val_df['CryoSleep'].map(binary_mapping)

train_cryo_sleep_most_frequent: False
test_cryo_sleep_most_frequent: False
val_cryo_sleep_most_frequent: False


/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/3489001263.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['CryoSleep'] = train_df['CryoSleep'].fillna(train_cryo_sleep_most_frequent)
/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/3489001263.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['CryoSleep'] = test_df['CryoSleep'].fillna(test_cryo_sleep_most_frequent)
/var/folders/n6/gd6z22dx3ts2dmwk_hjzk8lr0000gn/T/ipykernel_43760/3489001263.py:15: FutureWarning: Downcasting object dtype arrays on .f

In [66]:
### Transported only for TRAIN/TEST
train_transported_most_frequent = train_df['Transported'].value_counts().idxmax()
print(f"train_transported_most_frequent: {train_transported_most_frequent}")

test_transported_most_frequent = test_df['Transported'].value_counts().idxmax()
print(f"test_transported_most_frequent: {test_transported_most_frequent}")

### Fill all missing values with most frequent value
train_df['Transported'] = train_df['Transported'].fillna(train_transported_most_frequent)
test_df['Transported'] = test_df['Transported'].fillna(test_transported_most_frequent)

train_df['Transported'] = train_df['Transported'].map(binary_mapping)
test_df['Transported'] = test_df['Transported'].map(binary_mapping)

train_df.head()


train_transported_most_frequent: True
test_transported_most_frequent: True


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,cabin_letter
2333,1,0,1,28.0,0,0.0,55.0,0.0,656.0,0.0,0,6
2589,1,0,1,17.0,0,0.0,1195.0,31.0,0.0,0.0,0,6
8302,2,1,2,28.0,0,0.0,0.0,0.0,0.0,0.0,1,3
8177,3,0,1,20.0,0,NaN,2.0,289.0,976.0,0.0,1,6
500,2,1,2,36.0,0,0.0,0.0,0.0,0.0,0.0,1,3


### Train models

In [67]:
X_train = train_df.drop(columns=['Transported'])
y_train = train_df['Transported']

X_test = test_df.drop(columns=['Transported'])
y_test = test_df['Transported']


In [68]:
models = []

models.append(("xgb",XGBClassifier()))
models.append(("xgbgblinear",XGBClassifier(booster="gblinear")))
models.append(("LGBM",LGBMClassifier()))

In [69]:
results = {}

for  name, model in models:
    skf = StratifiedKFold(n_splits = 5, random_state=None)
    cv_results = cross_val_score(model,X_train,y_train,cv=skf, scoring="accuracy")
    results[name]= (cv_results.mean(), cv_results.std())

print("name     results.mean     results.std")

for key,value in results.items():
    print(key,value)

[LightGBM] [Info] Number of positive: 2800, number of negative: 2763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000505 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 5563, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503326 -> initscore=0.013302
[LightGBM] [Info] Start training from score 0.013302
[LightGBM] [Info] Number of positive: 2800, number of negative: 2763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1373
[LightGBM] [Info] Number of data points in the train set: 5563, number of used features: 11
[LightGBM] [Info] [binary:

## Grid Search

In [70]:
from sklearn.model_selection import GridSearchCV

lgbmc = LGBMClassifier()


lgbm_params = {"n_estimators":[100,200,300], 
               "learning_rate":[0.01,0.05,0.1,0.3],
               "num_leaves":[20,50,80,100]}

grid_search = GridSearchCV (estimator = lgbmc,
                            param_grid = lgbm_params,
                            n_jobs=-1,
                            cv = 5,
                            scoring="accuracy",
                           error_score='raise')

grid_result = grid_search.fit(X_train, y_train)


final_model = lgbmc.set_params(**grid_result.best_params_)


[LightGBM] [Info] Number of positive: 2800, number of negative: 2763
[LightGBM] [Info] Number of positive: 2800, number of negative: 2763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001026 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 5563, number of used features: 11
[LightGBM] [Info] Number of positive: 2800, number of negative: 2763
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503326 -> initscore=0.013302
[LightGBM] [Info] Start training from score 0.013302
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002182 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 5563, number of used features: 11
[LightGBM] [Info] Aut

In [71]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, f1_score


#training the model
final_model.fit(X_train, y_train)

#predictions
y_pred = final_model.predict(X_test)


precision = precision_score(y_pred, y_test)
accuracy = accuracy_score(y_pred, y_test)
recall = precision_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test)

score = []
score.append(("precision", precision))
score.append(("accuracy",accuracy))
score.append(("recall",recall))
score.append(("f1",f1))

score= pd.DataFrame(score)
score.rename(columns={0: "Metric", 1:"Result"}, inplace=True)

display(score)

[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


,Metric,Result
0,precision,0.823462
1,accuracy,0.791259
2,recall,0.823462
3,f1,0.799337


## Test on Validation Data

In [ ]:
y_pred_val = final_model.predict(val_df)
display(y_pred_val)


4277


array([1, 0, 1, ..., 1, 1, 1], shape=(4277,))

In [84]:
val_df = pd.read_csv(VAL_FILE_PATH)

## Drop all columns except PassengerId and Transported
val_df = val_df.drop(columns=["HomePlanet", "Name", "Destination", "Cabin", "VIP", "CryoSleep", "Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"])

## Add Transported column
val_df['Transported'] = y_pred_val

## Map back 1 and 0 to True and False
val_df['Transported'] = val_df['Transported'].map({1: True, 0: False})

## Save to csv
val_df.to_csv("val_df.csv", index=False)






In [85]:
### Read Saved Data§
val_df = pd.read_csv("val_df.csv")
val_df.head()


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
